<a href="https://colab.research.google.com/github/anne030303/data-course-sample/blob/main/Week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import gzip, json
import datetime
import numpy as np

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-09 08:23:20--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.5’

All_Beauty.csv.5    100%[===================>]  14.78M  71.2MB/s    in 0.2s    

2022-01-09 08:23:20 (71.2 MB/s) - ‘All_Beauty.csv.5’ saved [15499476/15499476]

--2022-01-09 08:23:20--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.5’

meta_All_Beauty.jso 100%[===================>]   9.85M  --.-KB/s    in 0.1s   

In [3]:
# metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
# metadata['rank_num'] = pd.to_numeric(metadata['rank'].str.split('in', n = 1, expand = True)[0].str.replace(',',''))
# metadata['rank_cat'] = metadata['rank'].str.split('in', n = 1, expand = True)[1].str.replace('(','',regex=True)
# metadata['pricenum'] = pd.to_numeric(metadata['price'].str.replace('$','',regex=True), errors='coerce')
# metadata = metadata.drop(metadata[(metadata['rank_cat'] != ' Beauty & Personal Care ')].index)
# metadata_new = metadata[['asin', 'brand', 'title', 'pricenum', 'rank_num', 'description', 'also_view', 'also_buy']].copy()

In [5]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
ratings = ratings.sort_values("DATE", ascending=False).groupby(['reviewerID', 'asin']).head(1)
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [6]:
ratings_trainings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
334129,B012Z7IHHI,A29V7NPR0F03L3,5.0,1535673600,2018-08-31
365919,B01FIPS9WW,A27C3J238GH88L,5.0,1535673600,2018-08-31
357965,B01CW8SDLU,A353ECRVPE944M,1.0,1535673600,2018-08-31
336825,B014KEITCC,A2ZU9ALCPBPOVH,5.0,1535673600,2018-08-31
334128,B012Z7IHHI,A11UM0RQVSVR24,4.0,1535673600,2018-08-31


In [7]:
## 觀察測試資料中的使用者
# 測試資料的使用者數
print('測試資料的使用者數：',len(users))
# 過去曾經購買評論過的使用者在訓練集中的資料
ratings_trainings.loc[ratings_trainings['reviewerID'].isin(users)].head()

測試資料的使用者數： 584


,asin,reviewerID,overall,unixReviewTime,DATE
191532,B00W259T7G,A2BU6N5956T9NH,5.0,1533081600,2018-08-01
354624,B01BUNDOIA,A3JEQJCEMX20GI,5.0,1532649600,2018-07-27
240689,B01DKQAXC0,ATJRZUXNWAWIK,5.0,1529625600,2018-06-22
357239,B01COREXYA,A29834GBB4DOP1,5.0,1529107200,2018-06-16
279689,B007Z5RTRK,A3JEQJCEMX20GI,5.0,1526342400,2018-05-15


In [8]:
## 首先觀察近一個月被購買的商品特性
# 近一個月有被購買的商品的歷年評論紀錄
ratings_testitems_in_allrecords = ratings.loc[ratings['asin'].isin(ratings_testings['asin'].to_list())]
# 歷年紀錄轉成list
ratings_testitems_in_allrecords_list = ratings_testitems_in_allrecords.groupby('asin').agg(list).reset_index()
# 商品購買頻率(天數)(只買一次的不列入考慮)
ratings_testitems_in_allrecords_freq = ratings_testitems_in_allrecords.groupby('asin')['unixReviewTime'].agg(lambda x: np.nan if len(x) == 1 else ((max(x)-min(x))/(len(x)-1))/(3600*24)).sort_values()
# 最近一次購買商品距離2018-09-30的天數
ratings_testitems_in_allrecords_lastBuy = ratings_testitems_in_allrecords.groupby('asin')['DATE'].agg(lambda x: abs(max(x) - pd.Timestamp('2018-09-30'))).sort_values()
ratings_testitems_in_allrecords_initBuy = ratings_testitems_in_allrecords.groupby('asin')['DATE'].agg(lambda x: abs(min(x) - pd.Timestamp('2018-09-30'))).sort_values()

ratings_testitems_in_allrecords_asinIndex = ratings_testitems_in_allrecords_list.set_index('asin')
ratings_testitems_in_allrecords_asinIndex['times'] = ratings_testitems_in_allrecords.groupby('asin')['reviewerID'].agg(lambda x: len(x))
ratings_testitems_in_allrecords_asinIndex['freq'] = ratings_testitems_in_allrecords_freq
ratings_testitems_in_allrecords_asinIndex['lastBuy'] = ratings_testitems_in_allrecords_lastBuy
ratings_testitems_in_allrecords_asinIndex['initBuy'] = ratings_testitems_in_allrecords_initBuy
ratings_testitems_in_allrecords_asinIndex[['times','freq','lastBuy','initBuy']]
ratings_testitems_in_allrecords_asinIndex[['times','freq','lastBuy','initBuy']].sort_values('initBuy').head()

,times,freq,lastBuy,initBuy
asin,,,,
B00GCEJCYI,1,NaN,11 days,11 days
B01AE1TJV0,1,NaN,22 days,22 days
B01DOGOKNY,1,NaN,22 days,22 days
B015PWYO7W,1,NaN,28 days,28 days
B0064C8CY4,2,63.0,29 days,92 days


In [9]:
ratings_trainings_copy = ratings_trainings.copy().set_index('asin')
def rerange_overall(x):
    if x > 3:
        return 1
    elif x < 3:
        return 0
    else:
        return 0.5
# 將overall分數調整為喜歡、普通、不喜歡三項
ratings_trainings_copy.loc[:,'overall'] = ratings_trainings_copy['overall'].apply(lambda x: rerange_overall(x))
# 加入最近購買時間的分數
lastBuy = ratings_trainings_copy.groupby('asin')['DATE'].agg(lambda x: abs(max(x) - pd.Timestamp('2018-09-30')))
lastBuy_normal = ((lastBuy.max()-lastBuy)/(lastBuy.max()-lastBuy.min()))
ratings_trainings_copy['lastbuy'] = lastBuy_normal
# 加入購買頻率的分數，頻率越高越接近1，只購買過一次為0
freq = ratings_trainings_copy.groupby('asin')['unixReviewTime'].agg(lambda x: np.nan if len(x) == 1 else ((max(x)-min(x))/(len(x)-1))/(3600*24))#.sort_values()
freq_normal = ((freq.max()-freq)/(freq.max()-freq.min())).fillna(0)
ratings_trainings_copy['freq'] = freq_normal
# 加入購買總數的分數
times = ratings_trainings_copy.groupby('asin')['reviewerID'].agg(lambda x: len(x))
times_normal = (times.max()-times)/(times.max()-times.min())
ratings_trainings_copy['times'] = times_normal
# print(ratings_trainings.head())
ratings_trainings_copy = ratings_trainings_copy[['reviewerID','overall','lastbuy','freq','times']]
print(ratings_trainings_copy.head())

                reviewerID  overall  lastbuy      freq     times
asin                                                            
B012Z7IHHI  A29V7NPR0F03L3      1.0      1.0  0.999273  0.970653
B01FIPS9WW  A27C3J238GH88L      1.0      1.0  0.951581  0.999769
B01CW8SDLU  A353ECRVPE944M      0.0      1.0  0.943528  0.999884
B014KEITCC  A2ZU9ALCPBPOVH      1.0      1.0  0.987249  0.998151
B012Z7IHHI  A11UM0RQVSVR24      1.0      1.0  0.999273  0.970653


In [10]:
#設定各項分數及計算總分
weight = {'overall': 1, 'lastbuy': 1, 'freq': 1, 'times': 1}
ratings_trainings_copy['score'] = ratings_trainings_copy['overall']*weight['overall']+\
    ratings_trainings_copy['lastbuy']*weight['lastbuy']+\
    ratings_trainings_copy['freq']*weight['freq']+\
    ratings_trainings_copy['times']*weight['times']
ratings_trainings_copy = ratings_trainings_copy.reset_index()
ratings_trainings_copy.sort_values('score',ascending=False).head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,asin,reviewerID,overall,lastbuy,freq,times,score
8170,B01G00BHN2,A1NCD7336TI7JG,1.0,0.9997,0.997429,0.998382,3.995511
193,B01G00BHN2,A1S4TD298FONFT,1.0,0.9997,0.997429,0.998382,3.995511
20166,B01G00BHN2,AC3PHGRTVK3IY,1.0,0.9997,0.997429,0.998382,3.995511
5109,B01G00BHN2,A3A5UJSDYQW1TC,1.0,0.9997,0.997429,0.998382,3.995511
3441,B01G00BHN2,A188F761S9H6Y9,1.0,0.9997,0.997429,0.998382,3.995511


In [11]:
# 熱銷排行_依評論時間、頻率、次數、分數計算權重
ratings_topk1 = list(ratings_trainings_copy.groupby('asin')['score'].agg('mean').sort_values(ascending=False)[:100].keys())
#ratings_topk1

In [12]:
# 熱銷排行_近兩月評論數最高
ratings_top_reviewtimes = ratings_trainings[(ratings_trainings['DATE'] + datetime.timedelta(days = 60)) > '2018-09-01']
ratings_topk2 = list(ratings_top_reviewtimes['asin'].value_counts()[:100].to_dict().keys())
#ratings_topk2

In [13]:
print(ratings_trainings_copy.shape)
#減少資料量，取分數優於平均分的資料作訓練
ratings_trainings_topmean = ratings_trainings_copy.loc[ratings_trainings_copy['score'] > ratings_trainings_copy['score'].mean()]

(361012, 7)


In [14]:
top30 = int(ratings_trainings_copy.shape[0]*0.3)
ratings_trainings_top30 = ratings_trainings_copy.sort_values('score',ascending=False)[:top30]

In [15]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

# header: user_id,item_id,rating,timestamp

def recommender_UserBased(training_data, users=[], k=10, ratingstop=[]):

    # loading data from dataframe
    # user_to_items dict:
    # {
    #   'user': {
    #       'item': ratings...
    #   }...
    # }
    user_to_items = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])

        user_to_items[user][item] = rating

    #print("total users before filtering: ", len(user_to_items))

    # remove obscure user to decrease data size
    # filtering params
    remove_obscure_user = True
    user_rating_threshold = 3
    all_users = list(user_to_items.keys())
    for user in all_users:
        ratings = user_to_items[user]
        if remove_obscure_user and len(ratings) < user_rating_threshold:
            del user_to_items[user]

    #print("total users  after filtering: ", len(user_to_items))

    # generate item to user mapping dict
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for user, items in user_to_items.items():
        for item, rating in items.items():
            item_to_users[item][user] = rating

    # prepare data of computing user similarity 
    init_sim = lambda: [0 for _ in range(3)]
    factory = lambda: defaultdict(init_sim)
    pre_user_similarity = defaultdict(factory)
    n = len(item_to_users)
    index = 0
    for item, user_ratings in item_to_users.items():
        if len(user_ratings) > 1:
            # print(f"item: {item} have been rated by {len(user_ratings)} users progress: {index}/{n}")
            for user1, user2 in combinations(user_ratings.keys(), 2):
                xy = user_ratings[user1] * user_ratings[user2]
                xx = user_ratings[user1] ** 2
                yy = user_ratings[user2] ** 2
                pre_user_similarity[user1][user2][0] += xy
                pre_user_similarity[user1][user2][1] += xx
                pre_user_similarity[user1][user2][2] += yy

                pre_user_similarity[user2][user1][0] += xy
                pre_user_similarity[user2][user1][1] += xx
                pre_user_similarity[user2][user1][2] += yy
        index += 1

    user_similarity = {}
    for src_user in pre_user_similarity:
        user_similarity_order = []
        for dst_user, val in pre_user_similarity[src_user].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(user_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    user_similarity_order.insert(i, (dst_user, similarity))
                    break
            else:
                user_similarity_order.append((dst_user, similarity))
        user_similarity[src_user] = user_similarity_order

    recommendation = {}
    for user in users:
        if user in user_similarity:
            sim_users = user_similarity[user]
            recommended_items = []
            recommended_items_set = set()
            user_have_rated = set(user_to_items[user])
            stop_recommend = False
            for sim_user, _ in sim_users:
                items_from_sim_user = sorted(list(user_to_items[sim_user].items()), key=lambda item: item[1])
                for item, _ in items_from_sim_user:
                    if item not in user_have_rated and item not in recommended_items_set:
                        recommended_items.append(item)
                        recommended_items_set.add(item)
                    if len(recommended_items) >= k:
                        stop_recommend = True
                        break
                if stop_recommend:
                    break
            if len(recommended_items) != k:
                needk = k - len(recommended_items)
                recommended_items += ratingstop[:needk]
            recommendation[user] = recommended_items
        else:
            recommendation[user] = ratingstop[:k]
    return recommendation


In [16]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

def recommender_ItemBased(training_data, users=[], k=10, ratingstop=[]):

    # loading data from dataframe
    # item_to_users dict:
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])
        item_to_users[item][user] = rating

    #print("data converted")

    user_to_items = defaultdict(dict)
    for item, rating_users in item_to_users.items():
        for user, rating in rating_users.items():
            user_to_items[user][item] = rating

    #print("data inverted")

    init_sim = lambda: [0, 0, 0]
    factory = lambda: defaultdict(init_sim)
    pre_item_similarity = defaultdict(factory)
    for user, items in user_to_items.items():
        if len(items) > 1:
            for i1, i2 in combinations(items.keys(), 2):
                xy = items[i1] * items[i2]
                xx = items[i1] ** 2
                yy = items[i2] ** 2
                pre_item_similarity[i1][i2][0] += xy
                pre_item_similarity[i1][i2][1] += xx
                pre_item_similarity[i1][i2][2] += yy

                pre_item_similarity[i2][i1][0] += xy
                pre_item_similarity[i2][i1][1] += xx
                pre_item_similarity[i2][i1][2] += yy

    #print("sim data prepared")

    item_similarity = {}
    for src_item in pre_item_similarity:
        item_similarity_order = []
        for dst_item, val in pre_item_similarity[src_item].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(item_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    item_similarity_order.insert(i, (dst_item, similarity))
                    break
            else:
                item_similarity_order.append((dst_item, similarity))
        item_similarity[src_item] = item_similarity_order

#     print(f"get {k} recommendation items for for user: {users}")

    recommendation = {}
    for user in users:
        items = []
        items_set = set()
        stop = False
        user_has_rated = set(user_to_items[user])
        for item in user_has_rated:
            if item in item_similarity:
                for sim_item, _ in item_similarity[item]:
                    # skip the item user has rated
                    if sim_item not in user_has_rated and sim_item not in items_set:
                        items.append(sim_item)
                        items_set.add(sim_item)
                    if len(items) >= k:
                        stop = True
                        break
                if stop:
                    break
        if len(items) != k:
            needk = k - len(items)
            items += ratingstop[:needk]
        recommendation[user] = items
    return recommendation    

In [17]:
!pip install surprise

In [18]:
# 利用套件 surprise 實作 collaborative filtering 方法
import time
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic

def recommender_surprise(training_data, users=[], k=10, user_based=False, algo=KNNBasic, ratingstop=[]):
    
    # 過濾重複的資料(移至整理資料階段)
#     training_data = (
#         training_data
#         .sort_values("DATE", ascending=False)
#         .groupby(['reviewerID', 'asin']).head(1)
#     )
    
    # 將資料讀取成 Surprise 所需的格式
    reader = Reader(rating_scale=(0, 5))
    training_data = training_data[['reviewerID', 'asin', 'overall']]
    data = Dataset.load_from_df(training_data, reader=reader)
    
    # 設定所使用的演算法及參數
    sim_options = {
        'name': 'cosine',
        'user_based': user_based  # compute similarities between items
    }
    algo_impl = algo(sim_options=sim_options)
    trainset = data.build_full_trainset()
    algo_impl.fit(trainset)
    
    # 獲取推薦結果
    recommendation = {}
    for user in users:
        items_user_rated = set(training_data.loc[training_data['reviewerID'] == user]['asin'].to_list())
        recommend_item_list = []
        recommend_item_set = set()
        for item in items_user_rated:
            iid = algo_impl.trainset.to_inner_iid(item)
            recommend_items_iid = algo_impl.get_neighbors(iid, k)
            for sim_item_iid in recommend_items_iid:
                item_raw_id = algo_impl.trainset.to_raw_iid(sim_item_iid)
                if item_raw_id not in items_user_rated and item_raw_id not in recommend_item_set:
                    recommend_item_list.append(item_raw_id)
                    recommend_item_set.add(item_raw_id)

            if len(recommend_item_list) >= k:
                recommend_item_list = recommend_item_list[:k]
                break
        if len(recommend_item_list) != k:
            needk = k - len(recommend_item_list)
            recommend_item_list += ratingstop[:needk]
            #print(len(recommend_item_list))
        recommendation[user] = recommend_item_list

    return recommendation

In [19]:
# 結果評估
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

In [20]:
print('訓練集中分數前30% 搭配 經過權重計算分數的熱銷排行')
ratings_by_user = recommender_UserBased(ratings_trainings_top30, users, ratingstop = ratings_topk1)
print('UserBased',evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_ItemBased(ratings_trainings_top30, users, ratingstop = ratings_topk1)
print('ItemBased',evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_surprise(ratings_trainings_top30, users, ratingstop = ratings_topk1)
print('surprise',evaluate(ratings_testings_by_user, ratings_by_user))
print('訓練集中分數前30% 搭配 近兩月評論數最高的熱銷排行')
ratings_by_user = recommender_UserBased(ratings_trainings_top30, users, ratingstop = ratings_topk2)
print('UserBased',evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_ItemBased(ratings_trainings_top30, users, ratingstop = ratings_topk2)
print('ItemBased',evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_surprise(ratings_trainings_top30, users, ratingstop = ratings_topk2)
print('surprise',evaluate(ratings_testings_by_user, ratings_by_user))

訓練集中分數前30% 搭配 經過權重計算分數的熱銷排行
UserBased 0.003389830508474576
ItemBased 0.003389830508474576
Computing the cosine similarity matrix...
Done computing similarity matrix.
surprise 0.005084745762711864
訓練集中分數前30% 搭配 近兩月評論數最高的熱銷排行
UserBased 0.15084745762711865
ItemBased 0.15084745762711865
Computing the cosine similarity matrix...
Done computing similarity matrix.
surprise 0.15254237288135594


In [ ]:
# 全部的訓練集 搭配 經過權重計算分數的熱銷排行
ratings_by_user = recommender_UserBased(ratings_trainings_copy, users, ratingstop = ratings_topk1)
print(evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_ItemBased(ratings_trainings_copy, users, ratingstop = ratings_topk1)
print(evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_surprise(ratings_trainings_copy, users, ratingstop = ratings_topk1)
print(evaluate(ratings_testings_by_user, ratings_by_user))

total users before filtering:  323489
total users  after filtering:  4793
0.003389830508474576
data converted
data inverted
sim data prepared
0.005084745762711864
Computing the cosine similarity matrix...


* recommender_UserBased
推薦分數：0.003389830508474576
* recommender_ItemBased
推薦分數：0.005084745762711864
* recommender_surprise
推薦分數：0.005084745762711864

In [ ]:
# 分數優於平均值的訓練集 搭配 經過權重計算分數的熱銷排行
ratings_by_user = recommender_UserBased(ratings_trainings_topmean, users, ratingstop = ratings_topk1)
print(evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_ItemBased(ratings_trainings_topmean, users, ratingstop = ratings_topk1)
print(evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_surprise(ratings_trainings_topmean, users, ratingstop = ratings_topk1)
print(evaluate(ratings_testings_by_user, ratings_by_user))

total users before filtering:  216635
total users  after filtering:  3022
0.003389830508474576
data converted
data inverted
sim data prepared
0.005084745762711864
Computing the cosine similarity matrix...


* recommender_UserBased
推薦分數：0.003389830508474576
* recommender_ItemBased
推薦分數：0.005084745762711864
* recommender_surprise
推薦分數：0.005084745762711864

In [ ]:
# 全部的訓練集 搭配 近兩月評論數最高的熱銷排行
ratings_by_user = recommender_UserBased(ratings_trainings_copy, users, ratingstop = ratings_topk2)
print(evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_ItemBased(ratings_trainings_copy, users, ratingstop = ratings_topk2)
print(evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_surprise(ratings_trainings_copy, users, ratingstop = ratings_topk2)
print(evaluate(ratings_testings_by_user, ratings_by_user))

0.15084745762711865
0.14915254237288136
Computing the cosine similarity matrix...


* recommender_UserBased
推薦分數：0.15084745762711865
* recommender_ItemBased
推薦分數：0.14915254237288136
* recommender_surprise
推薦分數：0.14745762711864407

In [ ]:
# 分數優於平均值的訓練集 搭配 近兩月評論數最高的熱銷排行
ratings_by_user = recommender_UserBased(ratings_trainings_topmean, users, ratingstop = ratings_topk2)
print(evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_ItemBased(ratings_trainings_topmean, users, ratingstop = ratings_topk2)
print(evaluate(ratings_testings_by_user, ratings_by_user))
ratings_by_user = recommender_surprise(ratings_trainings_topmean, users, ratingstop = ratings_topk2)
print(evaluate(ratings_testings_by_user, ratings_by_user))

0.15084745762711865
0.15254237288135594
Computing the cosine similarity matrix...


* recommender_UserBased
推薦分數：0.15084745762711865
* recommender_ItemBased
推薦分數：0.15254237288135594
* recommender_surprise
推薦分數：0.15254237288135594